In [1]:
import os
import matplotlib.pyplot as plt
import soundfile as sf

import torch
from models.AGC_endtoend_r1.model import AGC_STFT_GRU

import io
import numpy as np

model = AGC_STFT_GRU(160, 40, 160, 80)
model = model.to('cpu')
model.eval()

state_dict = torch.load('/home/yhjeon/projects/IITP_SE/NS_AGC/models/AGC_endtoend_r1/trained_ckpts/default.pth.tar', map_location='cpu')
model.load_state_dict(state_dict['state_dict'])
model = model.to('cpu')
x, _ = sf.read('/home/yhjeon/projects/IITP_SE/NS_AGC/data/SITEC_AGC/test/distorted/distorted_2_29464_19759_0.21687061174858996.wav')
x = torch.FloatTensor(x)
x = x.to('cpu').unsqueeze(0)
print(x.shape)

hidden = torch.zeros([1, 1, 40])

torch_out, hidden_out = model(x, hidden)



# torch.onnx.export(model, x, "test.onnx", export_params=True, opset_version=10, do_constant_folding=True, input_names=['input'], 
#                   output_names=['output'], dynamic_axes={'input' : {0: 'batch_size', 1:'seq_len'}, 'output':{0: 'batch_size', 1:'seq_len'}})

torch.Size([1, 47199])


/home/yhjeon/anaconda3/envs/denoiser2/lib/python3.8/site-packages/torch/functional.py:650: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at ../aten/src/ATen/native/SpectralOps.cpp:863.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]


In [2]:
input_names = ['input', 'input_hidden']
output_names = ['output', 'output_hidden']

torch.onnx.export(model, (x, hidden), "AGC_frame_h40.onnx", verbose=True, input_names=input_names, 
                  output_names=output_names, dynamic_axes={'input' : {0: 'batch_size', 1:'seq_len'}, 'output':{0: 'batch_size', 1:'seq_len'}, 'input_hidden':{0: 'batch_size'}, 'output_hidden':{0: 'batch_size'}})


Exported graph: graph(%input : Float(*, *, strides=[47199, 1], requires_grad=0, device=cpu),
      %input_hidden : Float(*, 1, 40, strides=[40, 40, 1], requires_grad=0, device=cpu),
      %lin.bias : Float(1, strides=[1], requires_grad=1, device=cpu),
      %onnx::GRU_154 : Float(1, 120, 81, strides=[9720, 81, 1], requires_grad=0, device=cpu),
      %onnx::GRU_155 : Float(1, 120, 40, strides=[4800, 40, 1], requires_grad=0, device=cpu),
      %onnx::GRU_156 : Float(1, 240, strides=[240, 1], requires_grad=0, device=cpu),
      %onnx::MatMul_157 : Float(40, 1, strides=[1, 40], requires_grad=0, device=cpu)):
  %/Constant_output_0 : Long(device=cpu) = onnx::Constant[value={1}, onnx_name="/Constant"](), scope: models.AGC_endtoend_r1.model.AGC_STFT_GRU::
  %/Constant_1_output_0 : Long(1, strides=[1], requires_grad=0, device=cpu) = onnx::Constant[value={1}, onnx_name="/Constant_1"](), scope: models.AGC_endtoend_r1.model.AGC_STFT_GRU:: # /home/yhjeon/projects/IITP_SE/NS_AGC/models/AGC_endtoend_

/home/yhjeon/anaconda3/envs/denoiser2/lib/python3.8/site-packages/torch/onnx/symbolic_opset9.py:4662: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with GRU can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(


In [3]:
import onnx

onnx_model = onnx.load("AGC_frame_h40.onnx")
onnx.checker.check_model(onnx_model)

In [5]:
import onnxruntime

ort_session = onnxruntime.InferenceSession("AGC_frame_h40.onnx")

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# ONNX 런타임에서 계산된 결과값
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x), ort_session.get_inputs()[1].name: to_numpy(hidden)}
ort_outs = ort_session.run(None, ort_inputs)

# ONNX 런타임과 PyTorch에서 연산된 결과값 비교
np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)
np.testing.assert_allclose(to_numpy(hidden_out), ort_outs[1], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")



Exported model has been tested with ONNXRuntime, and the result looks good!
